In [2]:
import gymnasium as gym
import numpy as np
from stable_baselines3 import DQN, A2C
# from stable_baselines3.common.envs import DummyVecEnv

ImportError: cannot import name 'DummyVecEnv' from 'stable_baselines3.common.envs' (/Users/tek/dev/python/gmo-algo-trading/myenv/lib/python3.11/site-packages/stable_baselines3/common/envs/__init__.py)

In [ ]:
import gym
import numpy as np
from stable_baselines3 import DQN, A2C
from stable_baselines3.common.envs import DummyVecEnv

class TradingEnv(gym.Env):
    def __init__(self, prices):
        super(TradingEnv, self).__init__()
        self.prices = prices
        self.max_steps = len(prices)
        self.current_step = 0
        self.balance = 10000.0  # 初期資金
        self.held = 0  # 保有株数
        self.action_space = gym.spaces.Discrete(3)  # 買う、売る、何もしないの3つの行動
        self.observation_space = gym.spaces.Box(low=np.array([0, 0]), high=np.array([np.inf, np.inf]))  # 資金と保有株数を観測

    def step(self, action):
        current_price = self.prices[self.current_step]
        reward = 0

        if action == 0:  # 買う
            self.balance -= current_price
            self.held += 1
        elif action == 1 and self.held > 0:  # 売る
            self.balance += current_price
            self.held -= 1
            reward = self.balance  # 売却後の資金を報酬とする
        elif action == 2:  # 何もしない
            pass

        self.current_step += 1
        done = self.current_step == self.max_steps
        info = {}
        return np.array([self.balance, self.held]), reward, done, info

    def reset(self):
        self.current_step = 0
        self.balance = 10000.0
        self.held = 0
        return np.array([self.balance, self.held])